In [1]:
import pandas as pd
import numpy as np
import joblib  

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

# ===============================
# 1. Load Dataset
# ===============================

try:
    df = pd.read_csv('health_data.csv')
except FileNotFoundError:
    print(" Error: 'health_data.csv' not found. Please upload the dataset.")
    exit()

# ===============================
# 2. Feature Engineering (General)
# ===============================

# Date handling
df['ClaimDate'] = pd.to_datetime(df['ClaimDate'], format='mixed', errors='coerce')
df['ClaimYear'] = df['ClaimDate'].dt.year
df['ClaimMonth'] = df['ClaimDate'].dt.month
df['ClaimDay'] = df['ClaimDate'].dt.day
df['ClaimWeekday'] = df['ClaimDate'].dt.weekday

# Medication & Test counts
df['Num_Medications'] = df['Medications'].apply(
    lambda x: len(str(x).split(',')) if pd.notna(x) else 0
)
df['Num_Tests'] = df['Tests'].apply(
    lambda x: len(str(x).split(',')) if pd.notna(x) else 0
)

# Drop processed columns
df_processed = df.drop(columns=['Medications', 'Tests', 'ClaimDate'])

# ===============================
# 3. Encode Categorical Features
# ===============================
cat_cols = [
    'DiagnosisCode',
    'ProcedureCode',
    'PatientGender',
    'ProviderSpecialty',
    'PatientMaritalStatus',
    'PatientEmploymentStatus',
    'ProviderLocation',
    'ClaimType',
    'ClaimSubmissionMethod',
    'Visit_Reason'
]

label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_processed[col] = le.fit_transform(df_processed[col].astype(str))
    label_encoders[col] = le

# Target encoding
target_col = 'ClaimStatus'
le_target = LabelEncoder()
df_processed['ClaimStatus_Encoded'] = le_target.fit_transform(df_processed[target_col])

# --- SAVE ENCODERS ---
# We also add the target encoder to the dictionary so we can decode predictions later
label_encoders['ClaimStatus'] = le_target
joblib.dump(label_encoders, 'label_encoders.pkl') 
print("\n Label Encoders saved to 'label_encoders.pkl'")

print("Target Class Mapping:")
print(dict(zip(le_target.classes_, le_target.transform(le_target.classes_))))

# ===============================
# 3.5. Feature Engineering: Domain Specific (Task 2)
# ===============================
# This section adds the "Medical Relevance" and "Severity" features required by the assignment.

# --- Feature A: ICD-CPT Medical Relevance Score ---
# Logic: Calculate the probability of 'Approved' (Class 0) for each Diagnosis+Procedure pair.
# If a combo is historically always denied, it gets a low score.
df_processed['Is_Approved'] = (df_processed['ClaimStatus_Encoded'] == 0).astype(int)

relevance_map = df_processed.groupby(['DiagnosisCode', 'ProcedureCode'])['Is_Approved'].mean().reset_index()
relevance_map.rename(columns={'Is_Approved': 'ICD_CPT_Relevance_Score'}, inplace=True)

# if i have been diagonized with flue -> lipid test -> Approved ( Number) ( num of time rejected) -> 99/100 -> High score , reject -> low score 1/100 = 
# Merge score back into main dataframe
df_processed = df_processed.merge(relevance_map, on=['DiagnosisCode', 'ProcedureCode'], how='left')
df_processed['ICD_CPT_Relevance_Score'] = df_processed['ICD_CPT_Relevance_Score'].fillna(0)
df_processed.drop(columns=['Is_Approved'], inplace=True)

# --- Feature B: CPT Groupings ---
# Categorize Procedure codes into groups
num_codes = df_processed['ProcedureCode'].max()

def get_cpt_group(code):
    if code <= num_codes * 0.25:
        return 0 
    elif code <= num_codes * 0.50:
        return 1 
    elif code <= num_codes * 0.75:
        return 2 
    else:
        return 3 

df_processed['CPT_Group'] = df_processed['ProcedureCode'].apply(get_cpt_group)

# --- Feature C: Claim Severity Flag ---
# Logic: If claim amount is significantly higher than average, flag it as 'High Severity'
avg_claim = df_processed['ClaimAmount'].mean()
df_processed['Claim_Severity_Flag'] = (df_processed['ClaimAmount'] > avg_claim).astype(int)

print("\nAdded Medical Domain Features:")
print(df_processed[['DiagnosisCode', 'ProcedureCode', 'ICD_CPT_Relevance_Score', 'CPT_Group', 'Claim_Severity_Flag']].head())
# ===============================
# 4. Train / Validation / Test Split
# ===============================

train_df, temp_df = train_test_split(
    df_processed,
    test_size=0.3,
    random_state=42,
    stratify=df_processed['ClaimStatus_Encoded']
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    random_state=42,
    stratify=temp_df['ClaimStatus_Encoded']
)

# Save split datasets (Deliverable)
train_df.to_csv('train_data.csv', index=False)
val_df.to_csv('validation_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)

print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

# ===============================
# 5. Prepare Training Data
# ===============================

cols_to_drop = [
    'ClaimID',
    'PatientID',
    'ProviderID',
    'Reason_Code',
    'Reason_Description',
    'ClaimStatus',
    'ClaimStatus_Encoded'
]

X_train = train_df.drop(columns=cols_to_drop)
y_train = train_df['ClaimStatus_Encoded']

X_val = val_df.drop(columns=cols_to_drop)
y_val = val_df['ClaimStatus_Encoded']

X_test = test_df.drop(columns=cols_to_drop)
y_test = test_df['ClaimStatus_Encoded']

# ===============================
# 6. Grid Search (Train Only)
# ===============================

rf = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=3,
    verbose=2,
    n_jobs=1
)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# --- SAVE MODEL ---
joblib.dump(best_model, 'random_forest_model.pkl') # Saving Model
print("\n Best Model saved to 'random_forest_model.pkl'")

print("\nBest Hyperparameters:")
print(grid_search.best_params_)

# ===============================
# 7. Validation Evaluation
# ===============================

y_val_pred = best_model.predict(X_val)

val_accuracy = accuracy_score(y_val, y_val_pred)
val_f1 = f1_score(y_val, y_val_pred, average='weighted')

print("\n--- Validation Performance ---")
print(f"Accuracy: {val_accuracy:.4f}")
print(f"F1 Score (Weighted): {val_f1:.4f}")

# ===============================
# 8. Final Test Evaluation
# ===============================

y_test_pred = best_model.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_test_pred)

print("\n--- Test Performance ---")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score (Weighted): {test_f1:.4f}")

print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=le_target.classes_))

# ===============================
# 9. Feature Importance
# ===============================

feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importance.head(10))

# ===============================
# 10. Save Final Predictions (Requirement: Probability Score)
# ===============================
# Task 3 Requirement: "Provide probability score"

# Get the probability of the predicted class (Confidence)
y_probs = best_model.predict_proba(X_test)
# Take the maximum probability as the confidence score
confidence_scores = np.max(y_probs, axis=1)

# Create a final results dataframe
results_df = X_test.copy()
results_df['Actual_Status'] = le_target.inverse_transform(y_test)
results_df['Predicted_Status'] = le_target.inverse_transform(y_test_pred)
results_df['Confidence_Score'] = confidence_scores

# Save to CSV
results_df.to_csv('final_predictions_with_confidence.csv', index=False)
print("\nFinal predictions with probability scores saved to 'final_predictions_with_confidence.csv'")


 Label Encoders saved to 'label_encoders.pkl'
Target Class Mapping:
{'Approved': np.int64(0), 'Denied': np.int64(1), 'Pending': np.int64(2)}

Added Medical Domain Features:
   DiagnosisCode  ProcedureCode  ICD_CPT_Relevance_Score  CPT_Group  \
0           4390           2899                 0.000000          2   
1           3896           3311                 1.000000          2   
2           4490           2548                 0.076923          2   
3           3202           3111                 0.066667          2   
4            985           2499                 0.125000          2   

   Claim_Severity_Flag  
0                    0  
1                    1  
2                    1  
3                    1  
4                    0  
Train shape: (5511, 29)
Validation shape: (1181, 29)
Test shape: (1182, 29)
Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total ti

In [2]:
print(feature_importance.head(10))

                    Feature  Importance
19  ICD_CPT_Relevance_Score    0.447247
9          ProviderLocation    0.054715
6             PatientIncome    0.053313
1             DiagnosisCode    0.052765
2             ProcedureCode    0.050763
0               ClaimAmount    0.050173
3                PatientAge    0.041158
15                 ClaimDay    0.036969
12             Visit_Reason    0.034135
14               ClaimMonth    0.027692


In [3]:
# lets print Model Metrics
print("\n--- Test Performance ---")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score (Weighted): {test_f1:.4f}")

print("\nConfusion Matrix:")
print(conf_matrix)


--- Test Performance ---
Accuracy: 0.7530
F1 Score (Weighted): 0.7479

Confusion Matrix:
[[377  10   8]
 [ 31 218 125]
 [ 26  92 295]]


###  Part 1 Evaluation: Model Interpretation & Adjudication Logic

**1. Primary Driver of Adjudication (The "Why"):**
The model's decision-making is driven by the **ICD-CPT Relevance Score** (Importance: ~0.45). 
* **Logic:** This feature measures the historical probability of a specific Procedure (CPT) being approved for a specific Diagnosis (ICD). 
* **Business Impact:** The model has successfully learned medical necessity patterns. If a CPT code is rarely approved for a given diagnosis (low relevance score), the model learns to flag it as a "Rejection" with high confidence. This directly automates the detection of "unnecessary CPTs" as requested.

**2. Secondary Risk Factors:**
* **Provider Location (0.05):** The model identified specific provider locations as high-risk, likely correlating with regions having higher historical denial rates or fraud patterns.
* **Patient Income (0.05):** Demographics play a minor but noticeable role, potentially capturing correlations between income brackets and specific policy types or coverage limits.

**3. Conclusion:**
The Random Forest model is not guessing randomly; it is primarily adjudicating based on **medical relevance**. It effectively acts as a first-pass filter to auto-approve medically aligned claims (High Relevance Score) and auto-reject or flag claims where the procedure does not match the diagnosis history.